In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, RocCurveDisplay
from scipy.stats import bootstrap
from typing import List, Union

In [ ]:
def map_name(roc_dict):
    """
    Map new names for ROC curves with AUC and confidence intervals.
    
    Args:
        roc_dict (dict): Dictionary of ROC curve data, keyed by group name.
    
    Returns:
        dict: Mapping of old names to new names with AUC and CI.
    """
    name_map = {}
    for key, value in roc_dict.items():
        auc_ci = value["ci_auc"]
        new_name = f"{key} ({auc_ci[1]:.1f}%) 95% CI: {auc_ci[0]:.1f}%-{auc_ci[2]:.1f}%"
        name_map[key] = new_name
    return name_map

In [ ]:
def calc_auc(df, pred_col, true_col, group_col=None, acc=False, boot_n=2000):
    """
    Calculate AUC, its confidence interval, and optionally accuracy.
    
    Args:
        df (pd.DataFrame): Dataframe containing predictions and true labels.
        pred_col (str): Column name for predictions.
        true_col (str): Column name for true labels.
        group_col (str): Column name for grouping, if any.
        acc (bool): Whether to calculate accuracy.
        boot_n (int): Number of bootstrap iterations for CI calculation.
    
    Returns:
        dict: Dictionary containing the result table and ROC curves.
    """
    from sklearn.utils import resample
    
    roc_dict = {}
    if group_col is None:
        groups = ["AUC"]
        df["AUC"] = "Overall"
    else:
        groups = df[group_col].unique()

    result_auc = []
    for group in groups:
        if group_col:
            group_df = df[df[group_col] == group]
        else:
            group_df = df
        fpr, tpr, _ = roc_curve(group_df[true_col], group_df[pred_col])
        roc_auc = auc(fpr, tpr)

        # Bootstrap CI
        boot_results = bootstrap(
            data=(group_df[true_col].values, group_df[pred_col].values),
            statistic=lambda y, y_pred: auc(*roc_curve(y, y_pred)[:2]),
            confidence_level=0.95,
            n_resamples=boot_n,
            method="basic",
        )
        ci_auc = [boot_results.confidence_interval.low, roc_auc, boot_results.confidence_interval.high]
        
        roc_dict[group] = {"fpr": fpr, "tpr": tpr, "roc_auc": roc_auc, "ci_auc": ci_auc}
        
        row = {"group": group, "low": ci_auc[0], "auc": ci_auc[1], "high": ci_auc[2]}
        if acc:
            best_idx = np.argmax(tpr - fpr)
            accuracy = (tpr[best_idx] + 1 - fpr[best_idx]) / 2
            row.update({"low_acc": accuracy, "acc": accuracy, "high_acc": accuracy})
        result_auc.append(row)

    result_df = pd.DataFrame(result_auc)
    return {"table": result_df, "roc_dict": roc_dict}

def plot_roc(df, pred_col, true_col, group_col=None, fill=False, colors=None, boot_n=2000):
    """
    Plot ROC curves with optional confidence intervals.
    
    Args:
        df (pd.DataFrame): Dataframe containing predictions and true labels.
        pred_col (str): Column name for predictions.
        true_col (str): Column name for true labels.
        group_col (str): Column name for grouping, if any.
        fill (bool): Whether to show confidence intervals as filled areas.
        colors (list or dict): Colors for the ROC curves.
        boot_n (int): Number of bootstrap iterations for CI calculation.
    
    Returns:
        dict: Dictionary containing the plot and ROC data.
    """
    roc_data = calc_auc(df, pred_col, true_col, group_col=group_col, boot_n=boot_n)
    roc_dict = roc_data["roc_dict"]
    name_map = map_name(roc_dict)
    
    plt.figure(figsize=(8, 6))
    for group, data in roc_dict.items():
        plt.plot(
            data["fpr"], data["tpr"], 
            label=name_map[group], 
            color=colors[group] if colors and group in colors else None
        )
        if fill:
            fpr, tpr = data["fpr"], data["tpr"]
            auc_ci = data["ci_auc"]
            plt.fill_between(
                fpr, tpr - auc_ci[0], tpr + auc_ci[2], 
                color=colors[group] if colors and group in colors else "gray", 
                alpha=0.2
            )
    
    plt.plot([0, 1], [0, 1], "k--", label="Random")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.grid()
    plt.show()
    
    return {"plot": plt, "roc_data": roc_dict}
